In [2]:
import torch
from transformers import pipeline
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Veri setini yükleme
file_path = 'lemmatized_comments_with_labels.csv'
df = pd.read_csv(file_path)

# Lemmatized_Comment sütununu kullanarak tahmin yapma
comments = df['Lemmatized_Comment'].tolist()

# CUDA (GPU) kullanımını kontrol etme ve cihazı ayarlama
device = 0 if torch.cuda.is_available() else -1  # -1 CPU, 0 GPU anlamına gelir

# Zero-shot sınıflandırıcı modelini yükleme (Hugging Face pipeline kullanarak)
classifier = pipeline("zero-shot-classification", model="nlptown/bert-base-multilingual-uncased-sentiment", device=device)

# Olası etiketler
labels = ['positive', 'neutral', 'negative']

# Metni küçük parçalara bölme fonksiyonu (token sınırını aşmamak için)
def split_text(text, max_tokens=512):
    """Uzun metinleri bölmek için yardımcı fonksiyon."""
    words = text.split()
    chunks = [" ".join(words[i:i + max_tokens]) for i in range(0, len(words), max_tokens)]
    return chunks

# Zero-shot sınıflandırma işlemi
results = []
for comment in comments:
    if isinstance(comment, str):  # Eğer metin mevcutsa
        chunks = split_text(comment, max_tokens=128)  # Metni küçük parçalara böl
        chunk_results = []

        # Parçalar için sınıflandırma yapılır
        for chunk in chunks:
            result = classifier(chunk, candidate_labels=labels)
            chunk_results.append(result)

        # Parçaların skorlarına göre genel sentiment belirleme
        chunk_scores = {label: 0 for label in labels}
        for res in chunk_results:
            chunk_scores[res['labels'][0]] += res['scores'][0]  # En yüksek skoru al

        # En yüksek skoru veren etiketi seçme
        predicted_label = max(chunk_scores, key=chunk_scores.get)
        results.append(predicted_label)
    else:
        continue

# Tahminleri dataframe'e ekleme
df['Predicted_Label'] = results

# Performans metriklerini hesaplama
accuracy = accuracy_score(df['Label'], df['Predicted_Label'])
precision, recall, f1, _ = precision_recall_fscore_support(df['Label'], df['Predicted_Label'], labels=labels, average='weighted')

# Tahminlerle birlikte dataframe'i yeni bir CSV dosyasına kaydetme
output_file = 'lemmatized_comments_with_predictions.csv'
df.to_csv(output_file, index=False)

# Performans metriklerini yazdırma
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Çıktı dosyasının yolunu sağlama
output_file


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Accuracy: 0.6242
Precision: 0.5221
Recall: 0.6242
F1-score: 0.4914


'lemmatized_comments_with_predictions.csv'